# Math Agent Example
This notebook shows a simple LangChain agent using a calculator tool.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.tools import Calculator
llm = ChatOpenAI(model='gpt-4o-mini')
tools = [Calculator()]
prompt = 'You are a math agent. Use tools when needed.'
agent = create_react_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools)
executor.invoke({'input': 'What is 25 * 17?'})